In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

# (issue) with procedures table

* If ex. 4333 is in both dictionarys with procedure and diagnose label this will either be repeated or overwritten
* 331 is an example of a code repeated in procedure dictionary with different labels, so again wil both be repreated or overwritten

In [ ]:
df_proc = pd.read_csv('~/mimic-iii-clinical-database-1.4/PROCEDURES_ICD.csv.gz')
df_dis  = pd.read_csv('~/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz') 

In [ ]:
df_proc.head()

In [ ]:
df_dis.head()

# label the proedures with help of ICD9_CODES from bioportal

some of the codes in procedure table contain diseases codes as from the website

https://www.findacode.com/search/search.php?str=3601

we can find the codes and se that exp. 4441 is both in procedures and in diseases, but 360.1 is only in diseases which we have in the procedures table

also 374 is in different catgeory both in procedure and in disease depending on which volume of the icd9 is used

In [ ]:
ICD9_CODES = pd.read_csv('/home/sohaib/ICD9_DIS_CODE.csv')
ICD9_CODES = ICD9_CODES.astype(str)

ICD9_CODES.head()

In [ ]:
ICD9_CODES[ICD9_CODES['0'] == '360.1']

In [ ]:
ICD9_LABEL = {
    '001-999.99': 'DISEASES AND INJURIES',
    '00-99.99': 'PROCEDURES',
    'E000-E999.9':'SUPPLEMENTARY CLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING',
    'V01-V91.99':'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES',
    'T051': 'Event',
    'T071': 'Entity'}

In [ ]:
def getcode(code):
    # find row 
    row = ICD9_CODES[ICD9_CODES['0'].str.replace(".","") == str(code)]

    if row.empty:
        return 'ERROR'

    label = ICD9_LABEL[row['6'].values[0]]
    return label

In [ ]:
# map the data
df_proc['Type'] = df_proc['ICD9_CODE'].progress_map(lambda x: getcode(x))

In [ ]:
df_proc

In [ ]:
issue = df_proc[df_proc['Type'] == 'ERROR']

issue

In [ ]:
issue['Type'] = issue['ICD9_CODE'].progress_map(lambda x: getcode(x))

In [ ]:
issue[issue['Type'] == 'ERROR']

In [ ]:
ICD9_CODES = pd.read_csv('/home/sohaib/ICD9_SUPP_EXT_CODE')



In [ ]:
df_label = pd.read_csv('~/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz')
df_label = df_label[['ICD9_CODE', 'LONG_TITLE', 'SHORT_TITLE']]
df_label = df_label.astype(str)

#df = pd.read_csv('/home/sohaib/mimic-iii-clinical-database-1.4/DIAGNOSES_ICD.csv.gz')
#df = df.astype(str)
df = pd.merge(df, df_label, on='ICD9_CODE', how='left')

df.

In [ ]:
df[~df['SHORT_TITLE_y'].isnull()]

In [ ]:
df.to_csv('~/Labels_procedure.csv')

In [ ]:
df  = pd.read_csv('/home/sohaib/Labels_procedure.csv')

df[df['Type'] == 'ERROR']

In [ ]:
df.to_csv('~/Labels_diagnose.csv')

In [ ]:
def filtercode(row):

    line = ICD9_PROC_CODE[ICD9_PROC_CODE['0'].str.replace(".", "") == row['ICD9_CODE']]

    if not line.empty:
        # return value corresponding to superclass in row
        index = pd.Series(line.values[0]).last_valid_index()
        label_row = ICD9_LABEL[ ICD9_LABEL['Class ID'] == row[str(index)].values[0]]
        label = label_row.values[0][1]
        
        new_row = row
        new_row['Type'] = label
        df = df.append([new_row], ignore_index=True)

# map the data
df.progress_apply(lambda x: filtercode(x), axis=1)

In [ ]:
df

In [ ]:
MIMIC_ICD9_CODE = pd.DataFrame()

def filtercode(row):
        # ------------------------------- DIAG ------------------------------------------
        line = ICD9_DIAG_CODE[ICD9_DIAG_CODE['0'].str.replace(".", "") == row['ICD9_CODE']]

        if not line.empty:
            # return value corresponding to superclass in row
            index = pd.Series(line.values[0]).last_valid_index()
            label_row = ICD9_LABEL[ ICD9_LABEL['Class ID'] == row[str(index)].values[0]]
            label = label_row.values[0][1]
        
            row['Type'] = label
            if MIMIC_ICD9_CODE.empty:
                MIMIC_ICD9_CODE = pd.DataFrame([row])
            else:
                MIMIC_ICD9_CODE = MIMIC_ICD9_CODE.append([row], ignore_index=True)
        
        # ------------------------------- PROC -------------------------------------------
        line = ICD9_PROC_CODE[ICD9_PROC_CODE['0'].str.replace(".", "") == row['ICD9_CODE']]
    
        if not line.empty:
            # return value corresponding to superclass in row
            index = pd.Series(line.values[0]).last_valid_index()
            label_row = ICD9_LABEL[ ICD9_LABEL['Class ID'] == row[str(index)].values[0]]
            label = label_row.values[0][1]
            
            row['Type'] = label
            if MIMIC_ICD9_CODE.empty:
                MIMIC_ICD9_CODE = pd.DataFrame([row])
            else:
                MIMIC_ICD9_CODE = MIMIC_ICD9_CODE.append([row], ignore_index=True)
                
# map the data
df.progress_apply(lambda x: filtercode(x), axis=1)

In [ ]:
MIMIC_ICD9_CODE

In [ ]:
df.to_csv('~/DIAG_CODE.csv', index=False)

In [ ]:
df = pd.read_csv('/home/sohaib/DIAG_CODE.csv')

df[df['HADM_ID'] == 100003]

In [ ]:
## Issue how to combine mutiple andhow to handle duplicated values

df = df[df['HADM_ID'] == 199993]

In [ ]:
df

In [ ]:
gb = df.groupby('HADM_ID')

df = pd.DataFrame()

for name, group in tqdm(gb):
    
    df_new = pd.DataFrame([np.append(group['ICD9_CODE'], name)], columns=np.append(group['Type'].values, 'HADM_ID')) 
    #print("Init", df_new) 
    
    # remove duplicates
    df_rep = df_new.loc[:,df_new.columns.duplicated()]
    df_new = df_new.loc[:,~df_new.columns.duplicated()]   
    df_copy = df_new
    
    # add duplicates as new rows
    #print("new", df_new)
    #print("rep", df_rep)
    #print("copy", df_copy)
    
    for name, value in df_rep.iteritems():
        df_add = df_copy.copy()
        df_add[name] = value.values[0]
        df_new = df_new.append(df_add, ignore_index=True)
    
    #print("PROD", df_new)
    if df.empty:
        df = df_new
    else:
        df = df.append(df_new, ignore_index=True)
    #print("--- Done ---")

In [ ]:
df[~df['ERROR'].isnull()]

In [ ]:
df.to_csv('~/DIAG_ROWS', index=False)

In [ ]:
df = pd.read_csv('~/DIAG_ROWS')

In [ ]:
df